In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
from bm import meta_dataset
from bm import meta_preprocess
from pathlib import Path
import bm
import sys
import os
from hydra import initialize, compose
import hydra
hydra.core.global_hydra.GlobalHydra.instance().clear()

sys.argv=['self.py']

os.chdir(Path(bm.__file__).parent.parent)
print(os.getcwd())
with initialize(version_base="1.1", config_path="conf"):
    cfg = compose(config_name="config.yaml", overrides=['+HYDRA_FULL_ERROR=1'])
    raws, events, infos = meta_preprocess.main(cfg)

Hostname lukas-B550-AORUS-ELITE-AX-V2 not defined in /conf/study_paths/study_paths.yaml. Using default paths.
/home/lukas/miniforge3/envs/lavis-bm3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Hostname lukas-B550-AORUS-ELITE-AX-V2 not defined in /conf/study_paths/study_paths.yaml. Using default paths.


/home/lukas/projects/brainmagick
hello there good sir.
Opening raw data file /home/lukas/projects/brainmagick/cache/studies/gwilliams2022/01_session0_story0/meg-sr120-hp0-raw.fif...
Isotrak not found
    Range : 0 ... 47519 =      0.000 ...   395.992 secs
Ready.
Opening raw data file /home/lukas/projects/brainmagick/cache/studies/gwilliams2022/01_session0_story1/meg-sr120-hp0-raw.fif...
Isotrak not found
    Range : 0 ... 85919 =      0.000 ...   715.992 secs
Ready.
[Gwilliams2022Recording('01_session0_story0'), Gwilliams2022Recording('01_session0_story1')]
Opening raw data file /home/lukas/projects/brainmagick/cache/studies/gwilliams2022/01_session0_story0/meg-sr128-hp0-raw.fif...
Isotrak not found
    Range : 0 ... 50687 =      0.000 ...   395.992 secs
Ready.
Reading 0 ... 50687  =      0.000 ...   395.992 secs...
Opening raw data file /home/lukas/projects/brainmagick/cache/studies/gwilliams2022/01_session0_story1/meg-sr128-hp0-raw.fif...
Isotrak not found
    Range : 0 ... 91647 =  

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
0it [00:06, ?it/s]


KeyboardInterrupt: 

In [5]:
import torch
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import librosa

# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self", torch_dtype=torch.float16, attn_implementation="flash_attention_2")
 # model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self", torch_dtype=torch.float16, attn_implementation="flash_attention_2", output_hidden_states=True).to(device)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)


def get_audio_label(wav_path, start, duration):
    data_dir = Path('data/gwilliams2022/download')
    audio, rate = librosa.load(os.path.join(data_dir, wav_path), sr = 16000, offset=start, duration=duration)
    
    # Process the audio to extract input features
    input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_values.to(device)
    
    with torch.no_grad():
        outputs = model(input_features)

    audio_embeddings = outputs.last_hidden_state
    return audio_embeddings

/home/lukas/anaconda3/envs/brainmagick/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from bm import speech_embeddings

generate_embeddings = speech_embeddings.SpeechEmbeddings(**cfg)

def get_audio_label(*args, **kwargs):
    return generate_embeddings.get_audio_embeddings(*args, **kwargs)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import torch
import pandas as pd
from mne.io import Raw
from mne.time_frequency import Spectrum
import julius
import json
from collections import defaultdict

# events[1][events[1]["kind"] == 'word']

# print(raws[0].annotations[-1])

# print(raws[0][:,52.12:])

# raw = raws[0]
# event = events[0][events[0]["kind"] == 'word']
# word_event = event.iloc[0]
# start = word_event['start']
# end = start + word_event['duration']
# print(start, end)
# t_idx = raw.time_as_index([start, end])
# data, times = raw[:,t_idx[0]:t_idx[1]]
# print(event)
# print(torch.tensor(data).shape)


print(os.getcwd())
# X = []
# Y = []
# word_labels = []
subs = defaultdict(list)
for raw, event, info in zip(raws, events, infos):
    raw: Raw
    event: pd.DataFrame
     
    word_events: pd.DataFrame = event[event['kind'] == 'word']
    raw.annotations.to_data_frame().info()
    descs = [json.loads(desc.replace("'", "\"")) for desc in raw.annotations.description]
    starts = [desc['start'] for desc in descs if desc['kind'] == 'word']
    sub_id = info['subject_info']['his_id']

    x = []
    y = []
    w_lbs = []
    
    # word_events.info()
    for (i, word_event), start in zip(word_events.iterrows(), starts):
        # print(i)
        if i >= 100: 
            break
    
        duration, word_label, wav_path = word_event['duration'], word_event['word'], word_event['sound'] 

        wav_path = wav_path.lower()

        offset = 0.
        start = start + offset
        end = start + duration + offset
        n_fft = 120
        # print(start, end, wav_path, word_label)
        t_idxs = raw.time_as_index([start, end])
        data, times = raw[:, t_idxs[0]:t_idxs[1]] 
        if data.shape[-1] < n_fft:
           continue

        spectrums: Spectrum = raw.compute_psd(tmin=start, tmax=end, fmax=60, n_fft=n_fft)
        # spectrums.plot(picks="data", exclude="bads", amplitude=False)
        data, freqs = spectrums.get_data(return_freqs=True)
        assert len(freqs) == 8
        
        # print('should have 8 freq bins: ', len(freqs))
        # preprocessed_data = to_spectrum(data)
        # bands = julius.split_bands(torch.Tensor(data), n_bands=10, sample_rate=120)

        audio_label = get_audio_label(wav_path, start, duration)

        print(f'word: {word_label}, audio features: {torch.tensor(audio_label).shape}, PSD: {torch.tensor(data).shape}')
        x.append(data)
        y.append(audio_label)
        w_lbs.append(word_label)
    # X.append(x)
    # Y.append(y)
    # word_labels.append(w_lbs)

    subs[sub_id].append((x, y, w_lbs))

# print(X, Y, word_labels)

# for each, start + offset, start + duration + offset

/home/lukas/projects/brainmagick
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3134 entries, 0 to 3133
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   onset        3134 non-null   datetime64[ns]
 1   duration     3134 non-null   float64       
 2   description  3134 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 73.6+ KB
Effective window size : 0.120 (s)
word: Tara, audio features: torch.Size([14, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: stood, audio features: torch.Size([11, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: stock, audio features: torch.Size([18, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: still, audio features: torch.Size([19, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: waiting, audio features: torch.Size([20, 768]), PSD: torc

/tmp/ipykernel_43143/1088313681.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(f'word: {word_label}, audio features: {torch.tensor(audio_label).shape}, PSD: {torch.tensor(data).shape}')


word: from, audio features: torch.Size([8, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: scout, audio features: torch.Size([14, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: craft, audio features: torch.Size([14, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: to, audio features: torch.Size([7, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: appear, audio features: torch.Size([13, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: darkness, audio features: torch.Size([20, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: of, audio features: torch.Size([6, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: The, audio features: torch.Size([6, 768]), PSD: torch.Size([208, 8])
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6622 entries, 0 to 6621
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  

/tmp/ipykernel_43143/1088313681.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(f'word: {word_label}, audio features: {torch.tensor(audio_label).shape}, PSD: {torch.tensor(data).shape}')


Effective window size : 0.120 (s)
word: Roy, audio features: torch.Size([27, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: What, audio features: torch.Size([10, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: You, audio features: torch.Size([10, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: suck, audio features: torch.Size([19, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: Chad, audio features: torch.Size([15, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: said, audio features: torch.Size([17, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: He, audio features: torch.Size([8, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: wished, audio features: torch.Size([14, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: Roy, audio features: torch.Size([10, 768]), PSD: torch.Size([208, 8])
Effec

/tmp/ipykernel_43143/1088313681.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(f'word: {word_label}, audio features: {torch.tensor(audio_label).shape}, PSD: {torch.tensor(data).shape}')


word: Acres, audio features: torch.Size([25, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: called, audio features: torch.Size([18, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: me, audio features: torch.Size([9, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: on, audio features: torch.Size([6, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: Thursday, audio features: torch.Size([21, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: afternoon, audio features: torch.Size([36, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: just, audio features: torch.Size([19, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: like, audio features: torch.Size([8, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: seen, audio features: torch.Size([14, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 

/tmp/ipykernel_43143/1088313681.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(f'word: {word_label}, audio features: {torch.tensor(audio_label).shape}, PSD: {torch.tensor(data).shape}')


word: his, audio features: torch.Size([6, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: desk, audio features: torch.Size([13, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: pulled, audio features: torch.Size([10, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: chair, audio features: torch.Size([14, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: close, audio features: torch.Size([19, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: legs, audio features: torch.Size([15, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: love, audio features: torch.Size([8, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: forced, audio features: torch.Size([16, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120 (s)
word: windows, audio features: torch.Size([16, 768]), PSD: torch.Size([208, 8])
Effective window size : 0.120

In [24]:
subs = list(subs.values())

In [32]:
subs[0][0][2]

['Tara',
 'stood',
 'stock',
 'still',
 'waiting',
 'for',
 'first',
 'tiny',
 'gleam',
 'from',
 'scout',
 'craft',
 'to',
 'appear',
 'darkness',
 'of',
 'The']

In [17]:
from lavis import registry
# import lavis.models.belt3_models.belt_clip_mae import Clip_TemporalConformer2D
import lavis.models.belt3_models.belt_clip_mae
import torch
model_cls = registry.get_model_class("Temporal-Spatial2D-Conformer")    
# Tara torch.Size([14, 768]) torch.Size([208, 19])
batch_size = 2
input_sample = {
    'eeg': torch.rand(batch_size, 208, 19, 1),
    'label': torch.randint(0, 2, (batch_size,)),
    "epoch": 0,
    "iters": 0,
    "mask_ratio": 0.50,
}
    
    # 'pretrain_time_mask', 'pretrain_channel_mask','freq_band_mask_welch_0.5'
for mae_type in ['base']:
    model = model_cls.from_config(type=mae_type).cpu()
    print('>>>>>>>>>>>>>>> clf model using {}'.format(mae_type))
    print('build success')
    ########################
    # forwad
    print('>>>>>>>>>>>>>>>forward')
    output = model(input_sample)
    for k in output:
        try:
            print(k, output[k].size())
        except:
            print(k, type(output[k]))
    ########################
    # generate
    print('>>>>>>>>>>>>>>>generate')
    output = model.generate(input_sample)
    for k in output:
        try:
            print(k, output[k].size())
        except:
            print(k, type(output[k]))

Hello from /projects/SilSpeech/Dev/SlientSpeech/LAVIS-main/lavis/datasets/builders/__init__.py


/home/lukas/miniforge3/envs/lavis-bm3/lib/python3.8/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/home/lukas/miniforge3/envs/lavis-bm3/lib/python3.8/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore
/home/lukas/projects/LAVIS-main/lavis/models/belt2_models/vq/vector_quantize_pytorch.py:264: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/lukas/projects/LAVIS-main/lavis/models/belt2_models/vq/vector_quantize_pytorch.py:394: FutureWarning: `torch.cuda.amp.autocast(args..

initialize_weights
>>>>>>>>>>>>>>> clf model using base
build success
>>>>>>>>>>>>>>>forward


RuntimeError: adaptive_max_pool1d(): Expected input to have non-zero size for non-batch dimensions, but input has sizes [2, 1024, 0] with dimension 2 being empty

In [46]:


for i in range(10):
    print(torch.tensor(Y[0][i]).shape, torch.tensor(X[0][i]).shape, word_labels[0][i])

torch.Size([1, 14, 768]) torch.Size([208, 300]) Tara
torch.Size([1, 11, 768]) torch.Size([208, 240]) stood
torch.Size([1, 18, 768]) torch.Size([208, 370]) stock
torch.Size([1, 19, 768]) torch.Size([208, 400]) still
torch.Size([1, 20, 768]) torch.Size([208, 410]) waiting
torch.Size([1, 6, 768]) torch.Size([208, 130]) for
torch.Size([1, 4, 768]) torch.Size([208, 90]) the
torch.Size([1, 13, 768]) torch.Size([208, 270]) first
torch.Size([1, 15, 768]) torch.Size([208, 310]) tiny
torch.Size([1, 11, 768]) torch.Size([208, 240]) gleam


/tmp/ipykernel_21450/2237729784.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(torch.tensor(Y[0][i]).shape, torch.tensor(X[0][i]).shape, word_labels[0][i])


In [5]:
events[1]

,story,story_uid,sound_id,kind,start,sound,duration,filepath,phoneme,sequence_id,...,speech_rate,voice,pronounced,word,language,modality,word_sequence,phoneme_id,offset,uid
0,NaN,NaN,NaN,block,8.245,NaN,3.120000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,english,audio,NaN,NaN,NaN,The Cable Fort by Bill Glover
1,cable_spool_fort,1.0,0.0,word,8.245,stimuli/audio/cable_spool_fort_0.wav,0.130000,NaN,NaN,0.0,...,160.0,Ava,1.0,The,english,audio,The Cable Fort by Bill Glover,NaN,NaN,NaN
2,cable_spool_fort,1.0,0.0,sound,8.245,stimuli/audio/cable_spool_fort_0.0.wav,100.597778,/home/lukas/projects/brainmagick/data/gwilliam...,NaN,NaN,...,NaN,NaN,NaN,NaN,english,audio,NaN,NaN,0.0,NaN
3,cable_spool_fort,1.0,0.0,phoneme,8.245,stimuli/audio/cable_spool_fort_0.wav,0.050000,NaN,dh_B,0.0,...,160.0,Ava,1.0,NaN,english,audio,NaN,0.0,NaN,NaN
4,cable_spool_fort,1.0,0.0,phoneme,8.295,stimuli/audio/cable_spool_fort_0.wav,0.080000,NaN,ah_E,0.0,...,160.0,Ava,1.0,NaN,english,audio,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6729,cable_spool_fort,1.0,5.0,phoneme,700.556,stimuli/audio/cable_spool_fort_5.wav,0.070000,NaN,l_E,128.0,...,190.0,Allison,1.0,NaN,english,audio,NaN,1.0,NaN,NaN
6730,cable_spool_fort,1.0,5.0,word,700.626,stimuli/audio/cable_spool_fort_5.wav,0.430000,NaN,NaN,128.0,...,190.0,Allison,1.0,right,english,audio,It was all right,NaN,NaN,NaN
6731,cable_spool_fort,1.0,5.0,phoneme,700.626,stimuli/audio/cable_spool_fort_5.wav,0.090000,NaN,r_B,128.0,...,190.0,Allison,1.0,NaN,english,audio,NaN,0.0,NaN,NaN
6732,cable_spool_fort,1.0,5.0,phoneme,700.716,stimuli/audio/cable_spool_fort_5.wav,0.140000,NaN,ay_I,128.0,...,190.0,Allison,1.0,NaN,english,audio,NaN,1.0,NaN,NaN


In [4]:
import hydra.core.global_hydra
import bm
from bm import meta_dataset
import sys
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose, core
import hydra
from omegaconf import OmegaConf
from pathlib import Path
import inspect

hydra.core.global_hydra.GlobalHydra.instance().clear()

with initialize(version_base="1.1", config_path="../bm/conf"):
    cfg = compose(config_name="config.yaml")
    os.chdir(Path(bm.__file__).parent)
    print(cfg)
    raw, events = meta_dataset.main_nb(cfg)



# sys.argv = [sys.argv[0]]
# initialize(config_path="conf")
# cfg = compose(config_name="config.yaml")

# raw, events = meta_dataset.main()

Hostname lukas-B550-AORUS-ELITE-AX-V2 not defined in /conf/study_paths/study_paths.yaml. Using default paths.


{'wandb': {'use_wandb': False, 'project': 'brainmagick', 'group': 'brainmagick-group'}, 'num_prints': 5, 'device': 'cuda', 'num_workers': 5, 'verbose': 0, 'show': 0, 'download_only': False, 'slurm': {'mem_per_gpu': 100, 'time': 4320}, 'continue_sig': None, 'continue_best': True, 'seed': 2036, 'dummy': None, 'cache': './cache', 'features_models': './features_models', 'early_stop_patience': 10, 'eval_every': 1, 'eval_train_set': False, 'optim': {'name': 'adam', 'lr': 0.0003, 'beta2': 0.999, 'epochs': 200, 'batch_size': 64, 'loss': 'clip', 'use_weighting': False, 'max_batches': 1200, 'svd': 0.0, 'negatives': None, 'negative_pool_size': None}, 'clip': {'linear': None, 'twin': True, 'pool': False, 'tmin': None, 'tmax': None, 'tmin_train': None, 'tmax_train': None, 'center': False}, 'test': {'wer_negatives': 10000, 'wer_topx': 10, 'wer_random': False, 'wer_recordings': 40, 'wer_study': None}, 'dset': {'selections': ['gwilliams2022'], 'tmin': -0.5, 'tmax': 2.5, 'n_recordings': 4, 'n_subjects'

AssertionError: 